### 사용자 정의 함수

---
- 함수 기능 : 모델 학습 진행 함수
- 함수 이름 : training
- 매개변수  : 함수 구동시 필요한 재료
    - 학습을 위한 재료
        - model instance
        - learning datasets: feature와 label (학습전 tensor화)
        - loss function instance
        - optimization instanve
        - range of learning: epoch
        - 배치 크기: batch_size
        - 배치 개수: batch_cnt
        - validation datasets: feature & label (학습전 tensor화)
- 함수 결과 : 학습 시 epoch 마다 loss value와 performance index value
---

---
- 함수 기능 : epoch 단위 모델 학습 진행 함수
- 함수 이름 : epochTraining
- 매개변수  : 함수 구동 시 필요한 재료
    - 학습을 위한 재료
        - model instance
        - loss function instance
        - optimization instance
        - train datasets: feature & target (tpye tensor)
        - batch_size
        - batxh_nnt
- 함수 결과 : loaa value & performance index value
---

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optima

from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f" device: {DEVICE}")

In [9]:
# 검즘 및 테스트
def testing(model, feature, label, loss_f, is_regress=True):
    
    with torch.no_grad():
        X_val = torch.FloatTensor(feature.value).to(DEVICE)
        y_val = torch.FloatTensor(label.value).to(DEVICE)
        

        y_pred = model(X_val).to(DEVICE)

        loss = loss_f(y_pred, y_val)
        
        score = R2Score()(y_pred, y_val) if is_regress else F1Score()(y_pred, y_train)
        
    return loss, score

In [10]:
# epoch 단위 모델 학습 진행 함수
def epoch_training(model, feature, label, loss_f, optimizer, batch_size=32, is_regress=True):
    batch_cnt = feature.shape[0] // batch_size
    
    loss_total, score_total = 0, 0
    # epoch에서 batch_size 만큼 datasets 추출 후 학습 진행
    for batch in range(batch_cnt):
        start = batch * batch_size
        end = start + batch_size
    
        X_train = torch.FloatTensor(feature[start:end].value).to(DEVICE)
        y_train = torch.FloatTensor(label[start:end].value).to(DEVICE)
        
        # model learning
        y_pred = model(X_train).to(DEVICE)
        
        # loss calcu
        loss = loss_f(y_pred, y_train)
        loss_total += loss
        
         # performance index value
        # test 및 성능 결과값 함수로... 
        score = R2Score()(y_pred, y_train) if is_regress else F1Score()(y_pred, y_train)
        score_total += score
    
    
        # optimization: weight, bais update
        optimizer.zero_grad()
        loss_f.backward()
        optimizer.step()
        
    
    # return
    return loss_total/batch_cnt, score_total/batch_cnt

In [ ]:
feature_df = pd.DataFrame(data=np.array([[1, 2, 3], [1, 2, 2]]))
label_df = pd.DataFrame(data=np.array([[1, 13, 4]]))

X_train, X_test, y_train, y_test = train_test_split(feature_df, label_df)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)


model_instance = nn.Linear(1, 2)
loss_function = optima.Adam(model_instance.parameters(), lr=0.001)

In [ ]:
# 학습진행
EPOCHS = 1000
train_val_loss = {'train':[], 'val':[]}
train_val_score = {'train':[], 'val':[]}

for epoch in  range(EPOCHS):
    # 학습진행
    train_loss, train_score = epoch_training(model_instance, X_train, y_train, loss_function)
    
    # 검증 진행
    val_loss, val_score = testing(model_instance, X_val, y_val)
    
    train_val_loss['train'].append(train_loss.item())
    train_val_score['train'].append(train_score.item())
    train_val_loss['val'].append(val_loss.item())
    train_val_score['val'].append(val_loss.item())
    
    print(f"[{epoch:5}/{EPOCHS:5}]  [Train]      Loss: {train_loss.item():.8f}, Score: {train_score.item():.8f}")
    print(f"[{epoch:5}/{EPOCHS:5}]  [Validation] Loss: {val_loss.item():.8f}, Score: {val_score.item():.8f}")